<div align=center>
<font size=5>
<font/>
<p></p>
 <br/>

<font color=#FF7500>
Sharif University of Technology - Departmenet of Computer Engineering
</font>
<p></p>
<font color=blue>
Linear Algebra
</font>
<br/>
<br/>

Spring 2023

</div>

<hr/>
		<div align=center>
		    <font color=red size=6>
			    <br />
Practical Assignment 3
            	<br/>
			</font>
</font>
                <br/>
    </div>


Here’s the English translation of the question:

Question:

Imagine a multi-dimensional world where several spaceships are engaged in battle. Each spaceship is equipped with a device that transmits its position over time to the command station. To prevent other spaceships from tracking their position, this device performs several linear transformations before sending the position.

For example:

The original position of the spaceship:
$$\begin{bmatrix} 1 \\ 2 \\ -5 \end{bmatrix}$$

The linear transformation matrix of the device:
$$
\mathbf{A} = \begin{bmatrix} 
0 & 1 & 2 \\ 
1 & 2 & 0 \\ 
1 & 0 & 1 
\end{bmatrix}
$$
The transmitted position by the device:

$\begin{bmatrix} 0 & 1 & 2 \\ 1 & 2 & 0 \\ 1 & 0 & 1 \end{bmatrix}$ * $\begin{bmatrix} 1 \\ 2 \\ -5 \end{bmatrix}$ = $\begin{bmatrix} -8 \\ 5 \\ 4 \end{bmatrix}$

The command station also has the transformation matrices to decode the spaceship's position. Unfortunately, during one of the battles, your position transmission device breaks. Since you're familiar with linear algebra, you decide to manually transform the coordinates using another device that can only send messages to the command station. For this, you collect the spaceship's original coordinates and the transformations applied by the broken device.

Your goal is to find the transformation matrix using this information and transmit new coordinates.

From your linear algebra lessons, you recall that applying several linear transformations to a point is equivalent to a single transformation, and the transformation matrix is the product of the individual transformation matrices. Therefore, the task is to find a transformation matrix.

If matrix 𝑋 represents the original positions of the spaceship (as columns) and matrix 𝑌 represents the transformed positions sent by the device (as columns), then the problem boils down to finding matrix 𝐴 that satisfies the equation: $Y = AX$

Since matrix 
𝑋 is not necessarily square, you cannot easily solve for 𝐴. However, in linear algebra, you learned about the pseudo-inverse, which can solve such equations. It is proven that under certain conditions, matrix 𝐴 can be calculated using the following formula: 
$$A = YX^T(XX^T)^{-1}$$
 

If the inverse of $𝑋𝑋^𝑇$ exists, the matrix 𝐴 will be the desired solution. Interestingly, there may not always be an exact solution for matrix 𝐴, but the above relation provides the best possible 𝐴, which, when multiplied by 𝑋, gives values close to those in 𝑌.

Your task is to compute matrix 𝐴 given the initial spaceship coordinates and the transformations applied by the broken device, and then, for a new set of coordinates provided in the input, return the final transformed coordinates.


Here’s the English translation of the input description:

**Input:** 

The first line: The number of series of coordinates encrypted by the device, denoted by 𝑚.
The second line: The number of linear transformations applied by the device, denoted by 𝑛.
The third line: The number of queries (coordinates) that we will need to encrypt later, denoted by 𝑝.
In the following lines, for each series of encrypted coordinates, the original spaceship coordinates are given first, followed by 𝑛 lines of transformed coordinates generated by the device.
Afterward, 𝑝 lines follow, each containing the query point coordinates.

**Output:**
There are three possible output scenarios:

**First Case:** It is possible that the inverse of the matrix $𝑋𝑋^𝑇$ does not exist. In this case, it is sufficient to print just one line with the phrase: 

`The results are unknown.`

One instance when this may happen is when matrix 𝑋 is a tall matrix (more rows than columns).

**Second Case:** The inverse of the matrix $𝑋𝑋^𝑇$ exists, but due to noise in the device, the result of multiplying 𝐴 (found using the above formula) by 
𝑋 is not exactly equal to 𝑌. Keep in mind, as mentioned before, matrix 𝐴 in this form provides the optimal solution for the system in question. In this case, before printing the transformed query coordinates, print the phrase:

`The results are noisy.`

Third Case: No noise is detected, and the inverse exists. In this case, directly print the transformed query coordinates without any extra message.


In [40]:
import numpy as np
def get_inputs(test_input):
    test_case = test_input.split("\n")
    
    m = int(test_case[0])
    n = int(test_case[1])
    p = int(test_case[2])
    
    test_case = test_case[3:]
    
    x = []
    y = []
    q = []

    for i in range(m):
        mapping = map(int, test_case[i * (n+1)].split())
        tmp = list(mapping)
        x.append(tmp)
        for j in range(n):
            inp = test_case[i * (n+1) + j + 1]
            if not (j - n + 1):
                mapping = map(int, inp.split())
                tmp = list(mapping)
                y.append(tmp)

    test_case = test_case[-p:]
    for i in range(p):
        mapping = map(int, test_case[i].split())
        tmp = list(mapping)
        q.append(tmp)

    return x, y, q

In [41]:
def inverse_matrix(matrix, row):
    augmented = np.c_[matrix, np.eye(row)]
    index = 0
    first_iter = list(range(row - 1))
    for i in first_iter:
        pivot = augmented[i][index]
        condition = bool(pivot)
        if not condition:
            second_iter = list(range(i + 1, row))
            for k in second_iter:
                condition = bool(augmented[k][index])
                if condition:
                    augmented_copy1 = augmented[k][:]
                    augmented_copy2 = augmented[i][:]
                    augmented[i], augmented[k] = augmented_copy1, augmented_copy2
                    break

            pivot = augmented[i][index]
        third_iter = list(range(i + 1, row))
        for k in third_iter:
            tmp = augmented[k][index] / pivot
            tmp *= augmented[i]
            augmented[k] -= tmp

        index += 1

    index = row - 1
    first_iter = list(range(1, row))
    first_iter.reverse()
    for i in first_iter:
        second_iter = list(range(0, i))
        second_iter.reverse()
        for j in second_iter:
            tmp = augmented[j][index] / augmented[i][index]
            tmp *= augmented[i]
            augmented[j] -= tmp

        index -= 1

    first_iter = list(range(row))
    for i in first_iter:
        augmented[i] = augmented[i] / augmented[i][i]

    return augmented[:, row:]

In [42]:
def calculate(x, y, q):

    np_x = np.array(x)
    np_y = np.array(y)
    np_q = np.array(q)
    xt = np.transpose(np_x)
    yt = np.transpose(np_y)
    qt = np.transpose(np_q)
    
    arr = np.matmul(xt, np.transpose(xt))
    inverse = inverse_matrix(arr, arr.shape[0])

    flag = False
    np_dot = np.dot(arr, inverse)
    np_rnd = np.round(np_dot)
    np_eye = np.eye(inverse.shape[0])
    if not np.all(np_rnd == np_eye):
        flag = True

    xtt = np.transpose(xt)
    inverse = np.array(inverse)
    arr = np.matmul(xtt, inverse)

    if flag:
        print("The results are unknown")

    else:
        A = np.matmul(yt, arr)
        B = np.matmul(A, qt)
        res = np.sum(np.abs(A @ xt - yt))
        if not res <= 0.01:
            print("The results are noisy")
        B_col = np.column_stack(B)
        for i in B_col:
            i[np.abs(i) < 0.000001] = 0
            temp = np.round(i, 2)
            for j in temp:
                print(j, " ", end="")
            print()

In [43]:
def main(test_input):
    x, y, q = get_inputs(test_input)
    calculate(x, y, q)

Let's test the code!

In [44]:
test_input = '''3
2
3
1 2 3
3 5
8 -2
-10 2 3
-8 5
-3 -13
2 9 1
11 10
21 1
2 4 9
3 1 3
-2 2 2'''
main(test_input)

print("##############################")

test_input = '''4
2
3
1 2 3
3 5
8 -2
-10 2 3
-8 5
-3 -13
2 9 1
11 10
21 1
2 9 1
11 10
21 1
2 4 9
3 1 3
-2 2 2'''
main(test_input)

print("##############################")


test_input = '''3
2
3
1 2 3
3 5
8 -2
1 2 3
-8 5
-3 -13
2 9 1
11 10
21 1
2 4 9
3 1 3
-2 2 2'''
main(test_input)

19.0  -7.0  
8.0  0.0  
4.0  -4.0  
##############################
19.0  -7.0  
8.0  0.0  
4.0  -4.0  
##############################
The results are unknown
